# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

$Gini(D_i)=1-\Sigma_{j=1}^xP_j^2$

0.4591836734693877

In [7]:
def get_gini(df, label):
    
    y = df[label]
    total = len(y) # 전체 데이터의 개수
    gini = 1
    for cat in np.unique(y): # 각 카테고리별로 gini계수를 구한다.
        dat = df[df[label]==cat] # 현재 카테고리에 속한 데이터들
        gini -= (len(dat)/total)**2 # 전체 데이터에서 현재 카테고리에 속하는 비율을 제곱하여 1에서 빼준다.
        
    return gini    

In [8]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

[['youth'],

 ['middle_aged'],
 
 ['youth', 'middle_aged'],
 
 ['senior'],
 
 ['youth', 'senior'],
 
 ['middle_aged', 'senior']]

In [35]:
def get_binary_split(df, attribute):
    from itertools import combinations
    classes = set(np.unique(df[attribute]))
    result = []
    for i in range(1, len(classes)): # 전체 클래스들 중에서 1개 뽑는 경우, 2개 뽑는 경우, ... len(classes)-1개 뽑는 경우에 대해
        for j in combinations(classes, i):
            result.append(list(j))
    return result

In [36]:
get_binary_split(pd_data,"age")

[['middle_aged'],
 ['youth'],
 ['senior'],
 ['middle_aged', 'youth'],
 ['middle_aged', 'senior'],
 ['youth', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

{'youth': 0.3936507936507936,

 'middle_aged': 0.35714285714285715,
 
 'youth_middle_aged': 0.4571428571428572,
 
 'senior': 0.45714285714285713,
 
 'youth_senior': 0.35714285714285715,
 
 'middle_aged_senior': 0.39365079365079364}

In [51]:
def get_attribute_gini_index(df, attribute, label):
    result ={}
    binary_split = get_binary_split(df, attribute)
    for i in binary_split:
        df_true = df[df[attribute].isin(i)] # 현재 split에 해당하는 열들만 뽑는다.
        df_false = df[~df[attribute].isin(i)] # 현재 split에 해당하지 않는 열들만 뽑는다.
        weight = len(df_true) / len(df) # 가중치를 구한다.
        gini = weight*get_gini(df_true, label) + (1-weight)*get_gini(df_false, label)
        result["_".join(i)] = gini # join을 이용해 이름을 정해진 형식으로 맞춰준다.
    return result

In [52]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'middle_aged': 0.35714285714285715,
 'youth': 0.3936507936507936,
 'senior': 0.4571428571428571,
 'middle_aged_youth': 0.4571428571428571,
 'middle_aged_senior': 0.39365079365079364,
 'youth_senior': 0.35714285714285715}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [53]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [55]:
##문제1 답안
get_binary_split(pd_data, "income")

[['low'],
 ['medium'],
 ['high'],
 ['low', 'medium'],
 ['low', 'high'],
 ['medium', 'high']]

In [75]:
##문제2 답안
target = 'class_buys_computer'
def find_min_gini_att(df, target):
    candidates = {}
    for col_name in df.columns: # 각각의 column들에 대해 gini index를 구해보자.
        if col_name == target: # target column은 건너뛴다.
            continue
        items = get_attribute_gini_index(df, col_name, target).items()
        if items == None or len(items)==0:
            continue
        candidate = min(items) # 각각의 컬럼에서 가장 작은 gini index를 가지는 attribute를 고른다.
        candidates [(col_name, candidate[0])] = candidate[1]
#     print(candidates)
    return min(candidates.items(), key = lambda x : x[1]) # 그 중 가장 작은 것을 고른다.
find_min_gini_att(pd_data, target)

(('age', 'middle_aged'), 0.35714285714285715)

Target이 computer구매 여부일 때, gini index가 가장 작은 변수는 `age`의 `middle_aged`이다.

In [76]:
##문제3 답안
df_true = pd_data[pd_data.age == 'middle_aged']
df_false = pd_data[pd_data.age != 'middle_aged']
print(find_min_gini_att(df_true, target))
print(find_min_gini_att(df_false, target))

(('income', 'high'), 0.0)
(('student', 'no'), 0.31999999999999984)


In [66]:
df_true

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [67]:
df_false

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


* middled_age인 사람들은 모두 computer를 구매하였다. 따라서 **어떤 변수를 선택하더라도 gini index는 항상 0이다.**
* middled_age가 아닌 사람들에 대해서 gini index가 최소인 변수는 **`student`의 `no`이다.**